<a href="https://colab.research.google.com/github/sareenv/SOEN-6611-Project/blob/main/Measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Libraries

In [28]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from IPython.display import display # to dispay the contents of the data frames.

Change the Path

In [29]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/MEASUREMENT-PROJECT/DATA")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
df_restuarant = pd.read_csv('michelin_my_maps.csv')

In [31]:
original_size = df_restuarant.shape[0]
print("The original value of the dataset is {}".format(original_size))

The original value of the dataset is 6522


In [32]:
df_modified = df_restuarant.drop(["PhoneNumber"], axis=1)
df_modified = df_modified.drop(["Url"], axis=1)
df_modified = df_modified.drop(["WebsiteUrl"], axis=1)

In [33]:
df_modified["Cuisine"] = df_modified["Cuisine"].str.split(',')
df_modified = df_modified.apply(pd.Series.explode)

In [34]:
display(df_modified.head())

,Name,Address,Location,MinPrice,MaxPrice,Currency,Cuisine,Longitude,Latitude,Award
0,Inter Scaldes,"Zandweg 2, Kruiningen, 4416 NA, Netherlands",Kruiningen,165,380,EUR,Modern Cuisine,4.024108,51.457161,3 MICHELIN Stars
1,De Librije,"Spinhuisplein 1, Zwolle, 8011 ZZ, Netherlands",Zwolle,232,232,EUR,Modern Cuisine,6.098018,52.515364,3 MICHELIN Stars
2,Hof van Cleve,"Riemegemstraat 1, Kruishoutem, 9770, Belgium",Kruishoutem,295,410,EUR,Creative,3.509865,50.903950,3 MICHELIN Stars
3,Boury,"Rumbeeksesteenweg 300, Roeselare, 8800, Belgium",Roeselare,100,160,EUR,Creative French,3.140398,50.937447,3 MICHELIN Stars
4,Zilte,"Hanzestedenplaats 5, Antwerpen, 2000, Belgium",Antwerpen,169,330,EUR,Creative,4.404754,51.228828,3 MICHELIN Stars


In [35]:
total_records = df_modified.shape[0]
print("Total number of records are {}".format(total_records))

Total number of records are 8278


In [36]:
minPriceMissingCondition = pd.isnull(df_modified["MinPrice"])
maxPriceMissingCondition = pd.isnull(df_modified["MaxPrice"])
currencyMissingCondition = pd.isnull(df_modified["Currency"])
condition = minPriceMissingCondition | maxPriceMissingCondition | currencyMissingCondition
missing_records_df = df_modified[condition]
print("Total records with missing attributes are {} in count ".format(missing_records_df.shape[0]))

Total records with missing attributes are 8 in count 


In [37]:
minPriceMean = df_modified[condition == False]["MinPrice"].apply(lambda x: x.replace(',', '')).apply(lambda x: float(x))
maxPriceMean = df_modified[condition == False]["MaxPrice"].apply(lambda x: x.replace(',', '')).apply(lambda x: float(x))
non_missing = df_modified[condition == False]
non_missing_cnt = non_missing.shape[0]
total_records = df_modified.shape[0]
completeness = non_missing_cnt / total_records
print("The completeness of the dataset before pre-processing is {}".format(completeness))

The completeness of the dataset before pre-processing is 0.9990335829910606


In [38]:
def dataSetSplitter(df_local,dates):
  df_local["ConversionDate"]=np.nan
  df_local = df_local.sample(frac=1).reset_index(drop=True)
  # splitting into multiple df
  results = np.array_split(df_local, len(dates)) 
  for index, result in enumerate(results):
    result["ConversionDate"] = pd.to_datetime(dates[index])
  return results

# date format - day/month/yyyy
df_splitted = dataSetSplitter(df_modified,['02/02/2022', 
                                           '18/04/2022',
                                           '10/03/2022', 
                                           '20/07/2022'
                                           ])
display(df_splitted[1].head())

,Name,Address,Location,MinPrice,MaxPrice,Currency,Cuisine,Longitude,Latitude,Award,ConversionDate
2070,Jägerhof,"Brühlbleichestrasse 11, Sankt Gallen, 9000, Sw...",Sankt Gallen,70,170,CHF,Modern Cuisine,9.383395,47.430792,1 MICHELIN Star,2022-04-18
2071,Quai n°4,"quai Saint-Jacques 4, Ath, 7800, Belgium",Ath,38,110,EUR,Modern Cuisine,3.778414,50.631302,1 MICHELIN Star,2022-04-18
2072,L.A. Jordan,"Ketschauerhofstraße 1, Deidesheim, 67146, Germany",Deidesheim,175,205,EUR,Creative,8.185337,49.407885,1 MICHELIN Star,2022-04-18
2073,Terme,"via Madonna Assunta, Pigna, 18037, Italy",Pigna,35,36,EUR,Ligurian,7.668460,43.929970,Bib Gourmand,2022-04-18
2074,Pensiun Aldier,"Plaz 154, Sent, 7554, Switzerland",Sent,70,84,CHF,Swiss,10.337619,46.816932,Bib Gourmand,2022-04-18
